In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep # control the crawl rate to avoid hammering the servers with too many requests
from random import randint
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities # dealing with pop-up permissions
from selenium.webdriver.firefox.options import Options

In [2]:
# what i need to get from the reviews
# 1) stars (0 - 5) ## "data-comp="StarRating" <div aria-label="4 stars">
# 2) reviewer id ## <a href="users/Tarah84" ...> under <strong data-at="nickname">____</strong>
# 3) reviewer features ## two lines under "nickname"
# 4) purchased shade ## line after headline of the review ("<h3 class>")

reviewer_id = []
rating = []
review_subject = []
reviewer_feature = []
purchased_shade = []

cols = {
    "reviewer_id": reviewer_id, "rating": rating, "review_subject": review_subject,
    "reviewer_feature": reviewer_feature, "purchased_shade": purchased_shade
    }

fenty_dat = pd.DataFrame(cols)


In [3]:
geoBlocked = webdriver.FirefoxOptions()
geoBlocked.set_preference("geo.prompt.testing", True)
geoBlocked.set_preference("geo.prompt.testing.allow", False)
driver = webdriver.Firefox(options=geoBlocked)
driver.get("https://www.sephora.com/product/pro-filtr-soft-matte-longwear-foundation-P87985432?skuId=2164697")
sleep(3)
driver.execute_script("window.scrollTo(0, 3000)") 
sleep(2)
driver.execute_script("window.scrollTo(0, 4500)") 

In [ ]:
# preparing to click
# driver.find_elements_by_css_selector('div[data-comp="Pagination Flex StyledComponent BaseComponent "')

In [4]:
# list_of_reviews = driver.find_elements_by_css_selector('div[data-comp="Review StyledComponent BaseComponent "]')

# list_of_reviews[0].find_elements_by_css_selector('strong[data-at="nickname"]')[0].text
list_of_reviews = []

def append_new_data(data = fenty_dat, list_of_reviews = list_of_reviews):
    shade = ""
    review_subject = ""
    reviewer_feature = ""
    reviewer_id = ""
    rating = int()
    for i in list_of_reviews:
        
        if i.find_elements_by_css_selector('strong[data-at="nickname"]') != []:
            reviewer_id = i.find_elements_by_css_selector('strong[data-at="nickname"]')[0].text
        else:
            pass 
        
        if i.find_elements_by_css_selector('strong[data-at="nickname"]+span') != []:
            reviewer_feature = i.find_elements_by_css_selector('strong[data-at="nickname"]+span')[0].text
        else:
            pass

        if i.find_elements_by_css_selector('div[data-comp="StarRating "]') != []:
            rating = int(i.find_elements_by_css_selector('div[data-comp="StarRating "]')[0].get_attribute("aria-label")[0])
        else:
            pass

        if i.find_elements_by_css_selector('h3') != []:
            review_subject = i.find_element_by_css_selector('h3').text
        else:
            review_subject = None

        if i.find_elements_by_css_selector('img[src*="https://www.sephora.com/productimages/"]+span') != []:
            shade = i.find_elements_by_css_selector('img[src*="https://www.sephora.com/productimages/"]+span')[0].text
        else:
            shade = None

        new_col = {
            "reviewer_id": reviewer_id,
            "rating": rating,
            "review_subject": review_subject,
            "reviewer_feature": reviewer_feature,
            "purchased_shade": shade
        }
        # print(new_col)
        data = data.append(new_col,ignore_index=True)
    return data

In [5]:
for n in tqdm(range(0, 201)):
    list_of_reviews = driver.find_elements_by_css_selector('div[data-comp="Review StyledComponent BaseComponent "]')
    fenty_dat = append_new_data(data = fenty_dat, list_of_reviews=list_of_reviews)
    driver.find_element_by_css_selector('button[aria-label="Next"]').click()
    sleep(randint(2,6))

  0%|          | 0/201 [00:00<?, ?it/s]/var/folders/b8/qhvvp5cj1hj_nj6sgf68yv3r0000gn/T/ipykernel_97162/3532620609.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  list_of_reviews = driver.find_elements_by_css_selector('div[data-comp="Review StyledComponent BaseComponent "]')
/opt/homebrew/Caskroom/miniforge/base/envs/foundation/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/opt/homebrew/Caskroom/miniforge/base/envs/foundation/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
/var/folders/b8/qhvvp5cj1hj_nj6sgf6

In [ ]:
fenty_dat.to_csv('fenty_dat_0_to_300.csv')

In [6]:
for n in tqdm(range(201, 601)):
    list_of_reviews = driver.find_elements_by_css_selector('div[data-comp="Review StyledComponent BaseComponent "]')
    fenty_dat = append_new_data(data = fenty_dat, list_of_reviews=list_of_reviews)
    driver.find_element_by_css_selector('button[aria-label="Next"]').click()
    sleep(randint(2,6))

  0%|          | 0/400 [00:00<?, ?it/s]/var/folders/b8/qhvvp5cj1hj_nj6sgf68yv3r0000gn/T/ipykernel_97162/1467378922.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  list_of_reviews = driver.find_elements_by_css_selector('div[data-comp="Review StyledComponent BaseComponent "]')
/var/folders/b8/qhvvp5cj1hj_nj6sgf68yv3r0000gn/T/ipykernel_97162/1467378922.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('button[aria-label="Next"]').click()
100%|██████████| 400/400 [29:16<00:00,  4.39s/it]


In [7]:
fenty_dat.to_csv('fenty_dat_0_to_600.csv')

In [8]:
for n in tqdm(range(601, 1000)):
    list_of_reviews = driver.find_elements_by_css_selector('div[data-comp="Review StyledComponent BaseComponent "]')
    fenty_dat = append_new_data(data = fenty_dat, list_of_reviews=list_of_reviews)
    driver.find_element_by_css_selector('button[aria-label="Next"]').click()
    sleep(randint(2,6))

  0%|          | 0/399 [00:00<?, ?it/s]/var/folders/b8/qhvvp5cj1hj_nj6sgf68yv3r0000gn/T/ipykernel_97162/130244487.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  list_of_reviews = driver.find_elements_by_css_selector('div[data-comp="Review StyledComponent BaseComponent "]')
/var/folders/b8/qhvvp5cj1hj_nj6sgf68yv3r0000gn/T/ipykernel_97162/130244487.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('button[aria-label="Next"]').click()
100%|██████████| 399/399 [29:45<00:00,  4.48s/it]


In [9]:
fenty_dat.to_csv('fenty_dat_0_to_999.csv')